In [2]:
import pandas

from datetime import datetime

from pandas import Timestamp

from pandas.tseries.offsets import DateOffset
# import zip
rateios_celesc = pandas.read_csv(r"C:\Users\Luan - PLIN\Documents\CELESC\COMPILADO-CELESC\COMPILADO-CELESC\COMPILADO RATEIOS - CELESC.csv")

rateios_celesc.drop(["OBSERVAÇÃO"],axis=1, inplace=True)

rateios_celesc['PORCENTAGEM']=rateios_celesc['PORCENTAGEM'].apply(lambda x : float(x.replace(",",".").replace("%",""))/100)

rateios_celesc['BENEFICIÁRIA']=rateios_celesc['BENEFICIÁRIA'].astype(str)

rateios_celesc.rename({"GERADORA":"UC"},axis=1,inplace=True)


rateios_celesc['UC']=rateios_celesc['UC'].apply(str)

rateios_celesc["datetime"]=rateios_celesc["REFERÊNCIA"].apply(lambda x : datetime.strptime(x,"%d/%m/%Y"))

injecao_geradores = pandas.read_csv(r"C:\Users\Luan - PLIN\Documents\CELESC\COMPILADO-CELESC\COMPILADO-CELESC\Faturas PLIN - CELESC.csv")

injecao_geradores.rename({"ucGeradora":"UC","referencia":"REFERÊNCIA"},axis=1,inplace=True)

columns_maintain = ["UC", "REFERÊNCIA", "INJEÇÃO TP"]

injecao_geradores = injecao_geradores[columns_maintain]

injecao_geradores["INJEÇÃO TP"]=injecao_geradores["INJEÇÃO TP"].apply(lambda x : float(x)) 

injecao_geradores["UC"]=injecao_geradores["UC"].apply(str) 

injecao_geradores["datetime"]=injecao_geradores["REFERÊNCIA"].apply(lambda x : datetime.strptime(x,"%d/%m/%Y"))

leituras_celesc = pandas.read_csv(r"C:\Users\Luan - PLIN\Documents\CELESC\COMPILADO-CELESC\COMPILADO-CELESC\LEITURAS__Geradorasdeenergia_CELESC_Conta Corrente_Ativo_Erro de leitura_Vencidas_05-08-2024.csv")

leituras_celesc=leituras_celesc.rename({"uc":"UC","date_ref":"REFERÊNCIA","kwh_compensado":"COMPENSADO","kwh_consumed":"CONSUMIDO"},axis=1)

leituras_celesc["COMPENSADO"]=leituras_celesc["COMPENSADO"].apply(lambda x : float(abs(float(x))))

leituras_celesc.rename({"UC":"BENEFICIÁRIA"},axis = 1 ,inplace=True)

columns_maintain=["BENEFICIÁRIA","REFERÊNCIA","COMPENSADO","CONSUMIDO"]

# TRATAMENTO DE DADOS LEITURAS

leituras_celesc=leituras_celesc[columns_maintain]

leituras_celesc.rename({"UC":"BENEFICIÁRIA"},inplace=True)

leituras_celesc["BENEFICIÁRIA"]=leituras_celesc["BENEFICIÁRIA"].apply(str)

leituras_celesc["REFERÊNCIA"]=leituras_celesc["REFERÊNCIA"].apply(lambda x: "01/"+x)

leituras_celesc["datetime"]=leituras_celesc["REFERÊNCIA"].apply(lambda x : datetime.strptime(x,"%d/%m/%Y"))

rateios_celesc=rateios_celesc.set_index(["UC","datetime","REFERÊNCIA"])

injecao_geradores=injecao_geradores.set_index(["UC","datetime","REFERÊNCIA"])

leituras_celesc.set_index(['BENEFICIÁRIA',"datetime","REFERÊNCIA"],inplace=True)

# CRIAÇÃO DE LEITURAS

missing_idx = rateios_celesc.reset_index().set_index(["BENEFICIÁRIA","datetime","REFERÊNCIA"]).index.difference(leituras_celesc.index)

# Criar um DataFrame com os índices faltantes e preenchê-los com NaN ou um valor padrão
missing_rows = pandas.DataFrame(index=missing_idx, columns=leituras_celesc.columns)
missing_rows['COMPENSADO'] = 0  # ou algum valor padrão, como 0

# Adicionar as linhas faltantes ao df2
leituras_celesc = pandas.concat([leituras_celesc, missing_rows])


def injecao_cliente_usina(row):
    try:
        inj_usina = injecao_geradores["INJEÇÃO TP"].loc[row.name].values
        inj_cliente_usina = inj_usina * row["PORCENTAGEM"]
        return float(format(inj_cliente_usina[0],".0f"))
    
    except:
        return None
    
rateios_celesc["INJ_CLIENTE_USINA"] = rateios_celesc.apply(injecao_cliente_usina,axis=1)

rateios_celesc.sort_index()

leituras_celesc.sort_index()

injecao_geradores.sort_index()


def dict_leitura (row:pandas.Series):
    # print(row.name)
    try:
        usinas_dict = rateios_celesc.query("BENEFICIÁRIA  == '{uc}' & REFERÊNCIA == '{ref}'".format(uc=row.name[0],ref = row.name[2]))["INJ_CLIENTE_USINA"].droplevel('REFERÊNCIA').to_dict()
        return usinas_dict
    except: 
        # new_row = pandas.DataFrame({(row.name[0],row.name[1]-DateOffset(months=1),datetime.strftime(datetime.strptime(row.name[2],"%d/%m/%Y")-DateOffset(months=1),"%d/%m/%Y")):[0]})
        return 0    

leituras_celesc["INJEÇÃO_USINAS"] = leituras_celesc.apply(dict_leitura,axis=1)

def inj_total (row): 

    return sum(row[2].values())

leituras_celesc["INJ_TOTAL"] = leituras_celesc.apply(inj_total,axis=1)

leituras_celesc=leituras_celesc.drop(["CONSUMIDO"],axis=1)

leituras_celesc=leituras_celesc.sort_index(axis=0,level='datetime',ascending=True)


    


C:\Users\Luan - PLIN\AppData\Local\Temp\ipykernel_11656\393978234.py:75: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  leituras_celesc = pandas.concat([leituras_celesc, missing_rows])
C:\Users\Luan - PLIN\AppData\Local\Temp\ipykernel_11656\393978234.py:80: PerformanceWarning: indexing past lexsort depth may impact performance.
  inj_usina = injecao_geradores["INJEÇÃO TP"].loc[row.name].values
C:\Users\Luan - PLIN\AppData\Local\Temp\ipykernel_11656\393978234.py:109: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return sum(row[2].values())


In [229]:
missing_idx = rateios_celesc.reset_index().set_index(["BENEFICIÁRIA","datetime","REFERÊNCIA"]).index.difference(leituras_celesc.index)

# Criar um DataFrame com os índices faltantes e preenchê-los com NaN ou um valor padrão
missing_rows = pandas.DataFrame(index=missing_idx, columns=leituras_celesc.columns)
missing_rows['COMPENSADO'] = 0  # ou algum valor padrão, como 0

# Adicionar as linhas faltantes ao df2
df2 = pandas.concat([leituras_celesc, missing_rows])

# Exibir o DataFrame resultantedf2
df2

C:\Users\Luan - PLIN\AppData\Local\Temp\ipykernel_7468\2454606875.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df2 = pandas.concat([leituras_celesc, missing_rows])


,,,COMPENSADO,INJEÇÃO_USINAS,INJ_TOTAL
BENEFICIÁRIA,datetime,REFERÊNCIA,,,
28842813,2023-03-01,01/03/2023,1194.0,"{('56930779', 2023-03-01 00:00:00): 1194.0}",1194.0
43808907,2023-03-01,01/03/2023,1037.0,"{('56930779', 2023-03-01 00:00:00): 1038.0}",1038.0
1077562,2023-04-01,01/04/2023,1768.0,"{('56930779', 2023-04-01 00:00:00): 902.0}",902.0
16375225,2023-04-01,01/04/2023,1510.0,{},0.0
18093570,2023-04-01,01/04/2023,565.0,"{('56930779', 2023-04-01 00:00:00): 541.0}",541.0
...,...,...,...,...,...
7633327,2024-07-01,01/07/2024,0.0,NaN,NaN
909505,2024-07-01,01/07/2024,0.0,NaN,NaN
9321870,2024-07-01,01/07/2024,0.0,NaN,NaN


In [213]:
new_dict = {key: value / 1000 for key, value in row.items()}


,,,COMPENSADO,INJEÇÃO_USINAS,INJ_TOTAL
BENEFECIÁRIA,datetime,REFERÊNCIA,,,
28842813,2023-03-01,01/03/2023,1194.0,"{('56930779', 2023-03-01 00:00:00): 1194.0}",1194.0
43808907,2023-03-01,01/03/2023,1037.0,"{('56930779', 2023-03-01 00:00:00): 1038.0}",1038.0
1077562,2023-04-01,01/04/2023,1768.0,"{('56930779', 2023-04-01 00:00:00): 902.0}",902.0
16375225,2023-04-01,01/04/2023,1510.0,{},0.0
18093570,2023-04-01,01/04/2023,565.0,"{('56930779', 2023-04-01 00:00:00): 541.0}",541.0
...,...,...,...,...,...
6025463,2024-07-01,01/07/2024,1630.0,"{('55128588', 2024-07-01 00:00:00): nan}",NaN
6673007,2024-07-01,01/07/2024,1028.0,{},0.0
6953000,2024-07-01,01/07/2024,952.0,{},0.0


In [19]:
leituras_celesc.iloc[2][1]

C:\Users\Luan - PLIN\AppData\Local\Temp\ipykernel_11656\864669900.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  leituras_celesc.iloc[2][1]


{('56930779', Timestamp('2023-03-01 00:00:00')): 2138.0}

In [190]:
leituras_celesc.reset_index().set_index(["UC","REFERÊNCIA"])

,,datetime,COMPENSADO,INJEÇÃO_USINAS,INJ_TOTAL
UC,REFERÊNCIA,,,,
28842813,01/03/2023,2023-03-01,1194.0,"{('56930779', 2023-03-01 00:00:00): 1194.0}",1194.0
43808907,01/03/2023,2023-03-01,1037.0,"{('56930779', 2023-03-01 00:00:00): 1038.0}",1038.0
1077562,01/04/2023,2023-04-01,1768.0,"{('56930779', 2023-04-01 00:00:00): 902.0}",902.0
16375225,01/04/2023,2023-04-01,1510.0,{},0.0
18093570,01/04/2023,2023-04-01,565.0,"{('56930779', 2023-04-01 00:00:00): 541.0}",541.0
...,...,...,...,...,...
6025463,01/07/2024,2024-07-01,1630.0,"{('55128588', 2024-07-01 00:00:00): nan}",NaN
6673007,01/07/2024,2024-07-01,1028.0,{},0.0
6953000,01/07/2024,2024-07-01,952.0,{},0.0


In [82]:

list(leituras_celesc.query("UC  == '{beneficiaria}' & REFERÊNCIA == '{ref}'".format(beneficiaria = rateios_celesc.iloc[200][1], ref = rateios_celesc.iloc[200].name[2])).values) == []

C:\Users\Luan - PLIN\AppData\Local\Temp\ipykernel_7468\2387091775.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  list(leituras_celesc.query("UC  == '{beneficiaria}' & REFERÊNCIA == '{ref}'".format(beneficiaria = rateios_celesc.iloc[200][1], ref = rateios_celesc.iloc[200].name[2])).values) == []


True

In [163]:
merged_df = rateios_celesc.merge(leituras_celesc, how='left', left_index=True, right_index=True, indicator=True)

# Filtrar linhas onde _merge é "left_only" (presentes apenas em df1)
not_in_df2 = merged_df[merged_df['_merge'] == 'left_only']

In [168]:
not_in_df2

CLIENTE  \
UC       datetime   REFERÊNCIA                                       
56930779 2024-07-01 01/07/2024  MB VENTILACAO E ILUMINACAO NATURAL   
                    01/07/2024  MB VENTILACAO E ILUMINACAO NATURAL   
                    01/07/2024  MB VENTILACAO E ILUMINACAO NATURAL   
                    01/07/2024                     MAIARA REGUEIRA   
                    01/07/2024       40.537.695 ALEX SANDRO MORAES   
...                                                            ...   
57521651 2024-04-01 01/04/2024                GUSTAVO MAY MOLINARI   
                    01/04/2024        PAMELA GONCALVES DE OLIVEIRA   
                    01/04/2024               JORGE GOETTEN DE LIMA   
                    01/04/2024               LUCIA GOETTEN DE LIMA   
                    01/04/2024                    CRISTIAN BEPPLER   

                               BENEFICIÁRIA  PORCENTAGEM  INJ_CLIENTE_USINA  \
UC       datetime   REFERÊNCIA                                                
56930779 2024-07-01 01/07/2024     55815984       0.2632                NaN   
                    01/07/2024     55815976       0.0651                NaN   
                    01/07/2024     55781559       0.0327                NaN   
                    01/07/2024     43808907       0.0574                NaN   
                    01/07/2024     10976863       0.0286                NaN   
...                                     ...          ...                ...   
57521651 2024-04-01 01/04/2024     11013740       0.1644             1762.0   
                    01/04/2024     18377012       0.1079             1157.0   
                    01/04/2024      3796140       0.4625             4958.0   
                    01/04/2024     46833910       0.0617              661.0   
                    01/04/2024     46689429       0.0545              584.0   

                                COMPENSADO INJEÇÃO_USINAS  INJ_TOTAL  \
UC       datetime   REFERÊNCIA                                         
56930779 2024-07-01 01/07/2024         NaN            NaN        NaN   
                    01/07/2024         NaN            NaN        NaN   
                    01/07/2024         NaN            NaN        NaN   
                    01/07/2024         NaN            NaN        NaN   
                    01/07/2024         NaN            NaN        NaN   
...                                    ...            ...        ...   
57521651 2024-04-01 01/04/2024         NaN            NaN        NaN   
                    01/04/2024         NaN            NaN        NaN   
                    01/04/2024         NaN            NaN        NaN   
                    01/04/2024         NaN            NaN        NaN   
                    01/04/2024         NaN            NaN        NaN   

                                   _merge  
UC       datetime   REFERÊNCIA             
56930779 2024-07-01 01/07/2024  left_only  
                    01/07/2024  left_only  
                    01/07/2024  left_only  
                    01/07/2024  left_only  
                    01/07/2024  left_only  
...                                   ...  
57521651 2024-04-01 01/04/2024  left_only  
                    01/04/2024  left_only  
                    01/04/2024  left_only  
                    01/04/2024  left_only  
                    01/04/2024  left_only  

[1066 rows x 8 columns]

In [166]:
rateios_celesc

CLIENTE  \
UC       datetime   REFERÊNCIA                                       
56930779 2024-07-01 01/07/2024  MB VENTILACAO E ILUMINACAO NATURAL   
                    01/07/2024  MB VENTILACAO E ILUMINACAO NATURAL   
                    01/07/2024  MB VENTILACAO E ILUMINACAO NATURAL   
                    01/07/2024                     MAIARA REGUEIRA   
                    01/07/2024       40.537.695 ALEX SANDRO MORAES   
...                                                            ...   
57521651 2024-04-01 01/04/2024                GUSTAVO MAY MOLINARI   
                    01/04/2024        PAMELA GONCALVES DE OLIVEIRA   
                    01/04/2024               JORGE GOETTEN DE LIMA   
                    01/04/2024               LUCIA GOETTEN DE LIMA   
                    01/04/2024                    CRISTIAN BEPPLER   

                               BENEFICIÁRIA  PORCENTAGEM  INJ_CLIENTE_USINA  
UC       datetime   REFERÊNCIA                                               
56930779 2024-07-01 01/07/2024     55815984       0.2632                NaN  
                    01/07/2024     55815976       0.0651                NaN  
                    01/07/2024     55781559       0.0327                NaN  
                    01/07/2024     43808907       0.0574                NaN  
                    01/07/2024     10976863       0.0286                NaN  
...                                     ...          ...                ...  
57521651 2024-04-01 01/04/2024     11013740       0.1644             1762.0  
                    01/04/2024     18377012       0.1079             1157.0  
                    01/04/2024      3796140       0.4625             4958.0  
                    01/04/2024     46833910       0.0617              661.0  
                    01/04/2024     46689429       0.0545              584.0  

[1066 rows x 4 columns]

In [109]:
inserir_leituras_(rateios_celesc.iloc[20])

C:\Users\Luan - PLIN\AppData\Local\Temp\ipykernel_7468\2883884440.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value = list(leituras_celesc.query("UC == '{beneficiária}' & REFERÊNCIA == '{ref}'".format(beneficiária = row[1], ref = row.name[2])).values)
